# Adaptive video

*Fraida Fund ffund@nyu.edu*

This experiment explores the tradeoff between different metrics of video quality (average rate, interruptions, and variability of rate) in an adaptive video delivery system.

It should take about 60-120 minutes to run this experiment.

**Preqreuisites**: To run this experiment on [FABRIC](https://fabric-testbed.net/), you should have a FABRIC account and be part of a FABRIC project. You should have already configured your JupyterHub environment in FABRIC following the instructions [here](https://github.com/fabric-testbed/jupyter-examples/blob/master/fabric_examples/fablib_api/configure_environment/configure_environment.ipynb).

## Background

### Adaptive video

In general high-quality video requires a higher data rate than a lower-quality equivalent. Consider the following two video frames. The first shows a video encoded at 200kbps:

![](https://witestlab.poly.edu/blog/content/images/2016/02/dash-200.png)

Here’s the same frame at 500kbps, with noticeably better quality:

![](https://witestlab.poly.edu/blog/content/images/2016/02/dash-500.png)

For web services that want to share video with their users, this poses a dilemma - what quality level should they use to encode the video? If a video is low quality, it will stream without interruption even on a slow 3G cellular connection, but a user on a high speed fiber network may be unhappy with the video quality. Or, the video may be high quality, but then the slow connection would not be able to stream it without constant interruptions.

Fortunately, there is a solution to this dilemma: adaptive video. Instead of delivering exactly the same video to every user, adaptive video delivers video that is matched to the individual user’s network quality.

There are many different adaptive video products: Microsoft Smooth Streaming, Apple HTTP Live Streaming (HLS), Adobe HTTP Dynamic Streaming (HDS), and Dynamic Adaptive Streaming over HTTP (DASH). This experiment focuses on DASH, which is widely supported as an international standard.

To prepare a video for adaptive video streaming with DASH, the video file is first encoded into different versions, each having a different rate and/or resolution. These are called *representations* or media presentations. The representations of a video all have the same content, but they differ in quality.

Each of these is further subdivided in time into *segments* of equal lengths (e.g., four seconds).

![](https://witestlab.poly.edu/blog/content/images/2016/02/dash-stored.png)

The content server then stores all of the segments of all of the representations (as separate files). Alongside these files, the content server stores a manifest file, called the Media Presentation Description (MPD). This is an XML file that identifies the various representations, identifies the video resolution and playback rate for each, and gives the location of every segment in each representation.

With these preparations complete, a user can begin to stream adaptive video from the server!

Once the MPD and video files are in place, users can start requesting DASH video.

First, the user requests the MPD file. It parses the MPD file, learns what representations are available, and decides what representation to request for the first segment. It then retrieves that specific file using the URL given in the MPD.

The user’s device keeps a video buffer (at the application layer). As new segments are retrieved, they are placed in the buffer. As video is played back, it is removed from the buffer.

Each time a client finishes retrieving a file into the buffer, it makes a new decision as to what representation to get for the next segment.

For example, the client might request the following representations for the first four segments of video:

![](https://witestlab.poly.edu/blog/content/images/2016/02/dash-requested.png)

The cumulative set of decisions made by the client is called a decision policy. The decision policy is a set of rules that determine which representation to request, based on some kind of client state - for example, what the current download rate is, or how much video is currently stored in the buffer.

The decision policy is not specified in the DASH standard. Many decision policies have been proposed by researchers, each promising to deliver better quality than the next!

### DASH decision policies

The obvious policy to maximize video quality alone would be to always retrive segments at the highest quality level. However, with this policy the user is likely to experience rebuffering - when playback is interrupted and the user has to wait for more video to be downloaded. This occurs when the video is being played back (and therefore, removed from the buffer) faster than it is being retrieved - i.e., the playback rate is higher than the download rate - so the buffer becomes empty. This state, which is known as buffer starvation, is obviously something we wish very much to avoid.

To create a positive user experience for streaming video, therefore, requires a delicate balancing act.

-   On the one hand, increasing the video playback rate too much (so that it is higher than the download rate) causes the undesired rebuffers.
-   On the other hand, decreasing the video playback rate also decreases the user-perceived video quality.

Performing rate selection to balance rebuffer avoidance and quality optimization is an ongoing tradeoff. Different DASH policies may make different decisions about how to balance that tradeoff. Different DASH policies may also decide to use different pieces of information for decision making. For example:

-   A decision policy may decide to focus on download rate in its decision making - select the quality level for the next video segment according to the download rate from the previous segment(s).
-   Or, a decision policy may focus on buffer occupancy (how much video is already downloaded into the buffer, waiting to be played back?) If there is already a lot of video in the buffer, the decision policy can afford to be aggressive in its quality selection, since it has a cushion to protect it from rebuffering. On the other hand, if there is not much video in the buffer, the decision policy should be careful not to select a quality level that is too optimistic, since it is at high risk of rebuffering.

### Specific policies in this implementation

In this experiment, we will use an updated version of the DASH implementation developed for the following paper:

> P. Juluri, V. Tamarapalli and D. Medhi, “SARA: Segment aware rate adaptation algorithm for dynamic adaptive streaming over HTTP,” 2015 IEEE International Conference on Communication Workshop (ICCW), 2015, pp. 1765-1770, doi: 10.1109/ICCW.2015.7247436.

which you can browse on [Github](https://github.com/teaching-on-testbeds/AStream). It includes three DASH decision policies:

The “basic” policy is a rate-based policy that tries to keep the video rate at or below the current network data rate. You can see [the “basic” implementation here](https://github.com/teaching-on-testbeds/AStream/blob/master/dist/client/adaptation/basic_dash2.py).

The buffer-based rate adaptation (“netflix”) algorithm uses the estimated network data rate only during the initial startup phase. Otherwise, it makes quality decisions based on the buffer occupancy, i.e. tring to avoid an empty buffer which would cause the video to freeze. It is based on the algorithm described in the following paper:

> Te-Yuan Huang, Ramesh Johari, and Nick McKeown. 2013. Downton abbey without the hiccups: buffer-based rate adaptation for HTTP video streaming. In Proceedings of the 2013 ACM SIGCOMM workshop on Future human-centric multimedia networking (FhMN ’13). Association for Computing Machinery, New York, NY, USA, 9–14. https://doi.org/10.1145/2491172.2491179

You can see [the “Netflix” implementation here](https://github.com/teaching-on-testbeds/AStream/blob/master/dist/client/adaptation/netflix_dash.py).

Finally, the segment-aware rate adaptation (“SARA”) algorithm uses the actual size of the segment and data rate of the network to estimate the time it would take to download the next segment. Then, given the current buffer occupancy, it selects the best possible video quality while avoiding buffer starvation. It is described in

> P. Juluri, V. Tamarapalli and D. Medhi, “SARA: Segment aware rate adaptation algorithm for dynamic adaptive streaming over HTTP,” 2015 IEEE International Conference on Communication Workshop (ICCW), 2015, pp. 1765-1770, doi: 10.1109/ICCW.2015.7247436.

You can see [the “SARA” implementation here](https://github.com/teaching-on-testbeds/AStream/blob/master/dist/client/adaptation/weighted_dash.py).

## Reserve resources on FABRIC

In this section, we’ll reserve and configure resources on the FABRIC testbed.

### Configure environment

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager() 
conf = fablib.show_config()

### Define FABRIC configuration for adaptive video experiment

In [ ]:
slice_name="adaptive-video-" + fablib.get_bastion_username()

node_conf = [
 {'name': "romeo",   'cores': 2, 'ram': 4, 'disk': 10, 'image': 'default_ubuntu_20', 'packages': ['net-tools', 'iperf3', 'moreutils']}, 
 {'name': "juliet",  'cores': 2, 'ram': 4, 'disk': 10, 'image': 'default_ubuntu_20', 'packages': ['net-tools', 'iperf3', 'moreutils']}, 
 {'name': "router",  'cores': 2, 'ram': 4, 'disk': 10, 'image': 'default_ubuntu_20', 'packages': ['net-tools']}
]
net_conf = [
 {"name": "net_r", "subnet": "10.10.1.0/24", "nodes": [{"name": "romeo",  "addr": "10.10.1.100"}, {"name": "router", "addr": "10.10.1.10"}]},
 {"name": "net_j", "subnet": "10.10.2.0/24", "nodes": [{"name": "juliet", "addr": "10.10.2.100"}, {"name": "router", "addr": "10.10.2.10"}]},
]
route_conf = [
 {"addr": "10.10.1.0/24", "gw": "10.10.2.10", "nodes": ["juliet"]}, 
 {"addr": "10.10.2.0/24", "gw": "10.10.1.10", "nodes": ["romeo"]}
]
exp_conf = {'cores': sum([ n['cores'] for n in node_conf]), 'nic': sum([len(n['nodes']) for n in net_conf]) }

### Reserve resources

Now, we are ready to reserve resources!

First, make sure you don’t already have a slice with this name:

In [ ]:
try:
    slice = fablib.get_slice(slice_name)
    print("You already have a slice by this name!")
    print("If you previously reserved resources, skip to the 'log in to resources' section.")
except:
    print("You don't have a slice named %s yet." % slice_name)
    print("Continue to the next step to make one.")
    slice = fablib.new_slice(name=slice_name)

We will select a random site that has sufficient resources for our experiment:

In [ ]:
while True:
    site_name = fablib.get_random_site()
    if ( (fablib.resources.get_core_available(site_name) > 1.2*exp_conf['cores']) and
        (fablib.resources.get_component_available(site_name, 'SharedNIC-ConnectX-6') > 1.2**exp_conf['nic']) ):
        break

fablib.show_site(site_name)

Then we will add hosts and network segments:

In [ ]:
# this cell sets up the nodes
for n in node_conf:
    slice.add_node(name=n['name'], site=site_name, 
                   cores=n['cores'], 
                   ram=n['ram'], 
                   disk=n['disk'], 
                   image=n['image'])

In [ ]:
# this cell sets up the network segments
for n in net_conf:
    ifaces = [slice.get_node(node["name"]).add_component(model="NIC_Basic", 
                                                 name=n["name"]).get_interfaces()[0] for node in n['nodes'] ]
    slice.add_l2network(name=n["name"], type='L2Bridge', interfaces=ifaces)

The following cell submits our request to the FABRIC site. The output of this cell will update automatically as the status of our request changes.

-   While it is being prepared, the “State” of the slice will appear as “Configuring”.
-   When it is ready, the “State” of the slice will change to “StableOK”.

You may prefer to walk away and come back in a few minutes (for simple slices) or a few tens of minutes (for more complicated slices with many resources).

In [ ]:
slice.submit()

In [ ]:
slice.get_state()
slice.wait_ssh(progress=True)

### Configure resources

Next, we will configure the resources so they are ready to use.

In [ ]:
slice = fablib.get_slice(name=slice_name)

In [ ]:
# install packages
# this will take a while and will run in background while you do other steps
for n in node_conf:
    if len(n['packages']):
        node = slice.get_node(n['name'])
        pkg = " ".join(n['packages'])
        node.execute_thread("sudo apt update; sudo apt -y install %s" % pkg)

In [ ]:
# bring interfaces up and either assign an address (if there is one) or flush address
from ipaddress import ip_address, IPv4Address, IPv4Network

for net in net_conf:
    for n in net['nodes']:
        if_name = n['name'] + '-' + net['name'] + '-p1'
        iface = slice.get_interface(if_name)
        iface.ip_link_up()
        if n['addr']:
            iface.ip_addr_add(addr=n['addr'], subnet=IPv4Network(net['subnet']))
        else:
            iface.get_node().execute("sudo ip addr flush dev %s"  % iface.get_device_name())

In [ ]:
# prepare a "hosts" file that has names and addresses of every node
hosts_txt = [ "%s\t%s" % ( n['addr'], n['name'] ) for net in net_conf  for n in net['nodes'] if type(n) is dict and n['addr']]
for n in slice.get_nodes():
    for h in hosts_txt:
        n.execute("echo %s | sudo tee -a /etc/hosts" % h)

In [ ]:
# enable IPv4 forwarding on all nodes
for n in slice.get_nodes():
    n.execute("sudo sysctl -w net.ipv4.ip_forward=1")

In [ ]:
# set up static routes
for rt in route_conf:
    for n in rt['nodes']:
        slice.get_node(name=n).ip_route_add(subnet=IPv4Network(rt['addr']), gateway=rt['gw'])

In [ ]:
# enable nodes to access IPv4-only resources, such as Github,
# even if the control interface is IPv6-only
from ipaddress import ip_address, IPv6Address
for node in slice.get_nodes():
    if type(ip_address(node.get_management_ip())) is IPv6Address:
        node.execute('echo "DNS=2a00:1098:2c::1" | sudo tee -a /etc/systemd/resolved.conf')
        node.execute('sudo service systemd-resolved restart')
        node.execute('echo "127.0.0.1 $(hostname -s)" | sudo tee -a /etc/hosts')
        node.execute('sudo rm -f /etc/resolv.conf; sudo ln -sv /run/systemd/resolve/resolv.conf /etc/resolv.conf')

### Draw the network topology

The following cell will draw the network topology, for your reference. The interface name and addresses of each experiment interface will be shown on the drawing.

In [ ]:
l2_nets = [(n.get_name(), {'color': 'lavender'}) for n in slice.get_l2networks() ]
l3_nets = [(n.get_name(), {'color': 'pink'}) for n in slice.get_l3networks() ]
hosts   =   [(n.get_name(), {'color': 'lightblue'}) for n in slice.get_nodes()]
nodes = l2_nets + l3_nets + hosts
ifaces = [iface.toDict() for iface in slice.get_interfaces()]
edges = [(iface['network'], iface['node'], 
          {'label': iface['physical_dev'] + '\n' + iface['ip_addr'] + '\n' + iface['mac']}) for iface in ifaces]

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
plt.figure(figsize=(len(nodes),len(nodes)))
G = nx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(edges)
pos = nx.spring_layout(G)
nx.draw(G, pos, node_shape='s',  
        node_color=[n[1]['color'] for n in nodes], 
        node_size=[len(n[0])*400 for n in nodes],  
        with_labels=True);
nx.draw_networkx_edge_labels(G,pos,
                             edge_labels=nx.get_edge_attributes(G,'label'),
                             font_color='gray',  font_size=8, rotate=False);

### Log into resources

Now, we are finally ready to log in to our resources over SSH! Run the following cells, and observe the table output - you will see an SSH command for each of the resources in your topology.

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
slice_info = [{'Name': n.get_name(), 'SSH command': n.get_ssh_command()} for n in slice.get_nodes()]
pd.DataFrame(slice_info).set_index('Name')

Now, you can open an SSH session on any of the resources as follows:

-   in Jupyter, from the menu bar, use File \> New \> Terminal to open a new terminal.
-   copy an SSH command from the table, and paste it into the terminal. (Note that each SSH command is a single line, even if the display wraps the text to a second line! When you copy and paste it, paste it all together.)

You can repeat this process (open several terminals) to start a session on each resource. Each terminal session will have a tab in the Jupyter environment, so that you can easily switch between them.

## Set up the adaptive video experiment

Now, we’re going to install software and set up the materials we need specifically to transfer adaptive video across this network! You will do this by opening SSH sessions to each of the hosts in the topology, and running commands to set them up as needed.

Make sure you have the SSH commands ready for each of the hosts.

### Prepare the server

First, we will set up the “juliet” host as an adaptive video server. Open an SSH session on “juliet”, and run the commands in this section there.

At the server, we will set up an HTTP server which will serve the video files to the client.

First, install the Apache HTTP server:

``` bash
sudo apt update  
sudo apt install -y apache2  
```

Then, download the video segments and put them in the web server directory. This step will take some time - while it is running, you can open another tab and move on to configuration of the other hosts.

``` bash
wget https://nyu.box.com/shared/static/d6btpwf5lqmkqh53b52ynhmfthh2qtby.tgz -O media.tgz
sudo tar -v -xzf media.tgz -C /var/www/html/
```

The web server directory now contains 4-second segments of the “open” video clip [Big Buck Bunny](https://peach.blender.org/about/), encoded at different quality levels. The Big Buck Bunny DASH dataset is from:

> Stefan Lederer, Christopher Müller, and Christian Timmerer. 2012. Dynamic adaptive streaming over HTTP dataset. In Proceedings of the 3rd Multimedia Systems Conference (MMSys ’12). Association for Computing Machinery, New York, NY, USA, 89–94. DOI:https://doi.org/10.1145/2155555.2155570

### Prepare the router

Next, we will set up the router. Open an SSH session on “router”, and run the commands in this section there.

At the router, we will *emulate* different network conditions, to see how each DASH policy performs.

We will experiment with both a constant data rate, and a variable data rate like that experienced by a mobile user. For the mobile user, we’ll use some network traces collected in the New York City metro area. With these traces, the data rate experienced by the DASH client in our experiment will mimic the experience of traveling around NYC on bus, subway, and ferry.

The NYC traces are shared from the following paper:

> Lifan Mei, Runchen Hu, Houwei Cao, Yong Liu, Zifa Han, Feng Li & Jin Li. (2019, March). Realtime Mobile Bandwidth Prediction using LSTM Neural Networks. In International Conference on Passive and Active Network Measurement. Springer.

To download the traces, on the “router” node run:

``` bash
git clone https://github.com/NYU-METS/Main nyc-traces
```

To extract the trace files from their compressed archive, we will need to install an appropriate utility:

``` bash
sudo apt update
sudo apt install -y unrar-free
```

Then, run

``` bash
unrar nyc-traces/Dataset/Dataset_1.rar
```

We will also download a couple of utility scripts to help us set a constant data rate or vary the data rate on the network. On the “router” node, run

``` bash
wget https://raw.githubusercontent.com/teaching-on-testbeds/adaptive-video/main/rate-vary.sh -O ~/rate-vary.sh
```

and

``` bash
wget https://raw.githubusercontent.com/teaching-on-testbeds/adaptive-video/main/rate-set.sh -O ~/rate-set.sh
```

### Prepare the client

Finally, we need to prepare the “romeo” host as a video client. Open an SSH session on “romeo”, and run the commands in this section there.

We must install Python3 to run the DASH video client, and we will also install the video encoding utility `ffmpeg` so that we can reconstruct the video later:

``` bash
sudo apt update
sudo apt install -y python3 ffmpeg
```

Now we are ready to run our experiments! We will run four experiments: one with a constant bit rate, one with a constant bit rate and an interruption in middle, one where we compare adaptive video policies under similar network conditions, and one with a varying bit rate using the NYC traces.

## Execute constant bit rate experiment

For this section, you will need an SSH session on the “router” node and one on the “romeo” node.

On the “router”, set a constant bit rate of 1000 Kbits/second with

``` bash
bash rate-set.sh 1000Kbit
```

(The first time you run it, you may see an error referencing a problem deleting a `qdisc`, but you can safely ignore this error.)

Note: you can specify a data rate in Kbits/second using `Kbit` or in Mbits/second using `Mbit`.

Then, on the client (“romeo”), start the DASH player with the “basic” adaptation policy:

``` bash
python3 ~/AStream/dist/client/dash_client.py -m http://192.168.1.2/media/BigBuckBunny/4sec/BigBuckBunny_4s.mpd -p 'basic' -d
```

(Note: you can alternatively try `netflix` or `sara` as the DASH policy.)

Leave this running for a while. Then, you can interrupt the DASH client with Ctrl+C.

To understand the performance of the DASH policy, we can look at the logs produced by the client. These will be located inside a directory named `ASTREAM_LOGS` in your home directory on the “romeo” node. Use

``` bash
ls ~/ASTREAM_LOGS
```

to find these.

In the data analysis section, we will use these logs - specifically the one that begins with `DASH_BUFFER_LOG_` - to understand the video adaptation policy that was applied in this experiment. We will copy the file associated with *this* experiment to `~/ASTREAM_LOGS/DASH_BUFFER_LOG_last.csv` with

``` bash
cp $(ls -t1  ~/ASTREAM_LOGS/DASH_BUFFER_LOG_*  | head -n 1 ) ~/ASTREAM_LOGS/DASH_BUFFER_LOG-last.csv
```

Also reconstruct the video that was delivered to the client. Use

``` bash
suffix=$(ls -lt | grep "TEMP_" | head -n 1 | cut -f2 -d"_")
cd ~/TEMP_$suffix
rm -f ~/BigBuckBunny.mp4 # if it exists
cat BigBuckBunny_4s_init.mp4 $(ls -vx BigBuckBunny_*.m4s) > BigBuckBunny_tmp.mp4
ffmpeg -i  BigBuckBunny_tmp.mp4 -c copy ~/BigBuckBunny.mp4
```

to combine the video segments into a `BigBuckBunny.mp4` file in your home directory.

## Data analysis

After each experiment run (with different variations in experiment conditions!) we can see the video that was delivered to the client, and also see how the video client made its decisions.

First, run this cell to make sure your FABRIC configuration and slice configuration is loaded.

In [ ]:
import os
os.environ['FABRIC_RC_FILE']=os.environ['HOME']+'/work/fabric_config/fabric_rc'
os.environ['FABRIC_BASTION_SSH_CONFIG_FILE']=os.environ['HOME']+'/work/fabric_config/ssh_config'

from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()                     

SLICENAME=fablib.get_bastion_username() + "_adaptive_video"
slice = fablib.get_slice(name=SLICENAME)

Next, the following cell will retrieve the reconstructed video file so that you can play it back inside this notebook:

In [ ]:
slice.get_node("romeo").download_file("/home/fabric/work/BigBuckBunny.mp4", "/home/ubuntu/BigBuckBunny.mp4")
from IPython.display import Video
Video("/home/fabric/work/BigBuckBunny.mp4", embed=True)

Then, you can use the log files to find out how the video client made its decisions. In the following cell, fill in the `DASH_BUFFER_LOG` log file name associated with the instance of *your* experiment that you want to analyze.

In [ ]:
DASH_BUFFER_LOG="DASH_BUFFER_LOG-last.csv"
slice.get_node("romeo").download_file("/home/fabric/work/DASH_BUFFER_LOG.csv", "/home/ubuntu/ASTREAM_LOGS/" + DASH_BUFFER_LOG)

and use the following cell to create a visualization. In the following plot, the line shows the bit rate of each segment as it is played back over time, and the colored background indicates whether the client is playing video (light cyan) or buffering (light pink).

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

c = {'INITIAL_BUFFERING': 'violet', 'PLAY': 'lightcyan', 'BUFFERING': 'lightpink'}

dash = pd.read_csv("/home/fabric/work/DASH_BUFFER_LOG.csv")
dash = dash.loc[dash.CurrentPlaybackState.isin(c.keys() )]
states = pd.DataFrame({'startState': dash.CurrentPlaybackState[0:-2].values, 'startTime': dash.EpochTime[0:-2].values,
                        'endState':  dash.CurrentPlaybackState[1:-1].values, 'endTime':   dash.EpochTime[1:-1].values})


for index, s in states.iterrows():
  plt.axvspan(s['startTime'], s['endTime'],  color=c[s['startState']], alpha=1) 

plt.plot(dash[dash.Action!="Writing"].EpochTime, dash[dash.Action!="Writing"].Bitrate, 'kx:')
plt.title("Video rate (bps)");
plt.xlabel("Time (s)");

We can also visualize the buffer occupancy over time. In the following plot, the line shows the number of segments in the buffer over time, and the colored background indicates whether the client is playing video (light cyan) or buffering (light pink). When the buffer occupancy goes to zero, the will have to stop playing in order to retrieve more data into the buffer.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

c = {'INITIAL_BUFFERING': 'violet', 'PLAY': 'lightcyan', 'BUFFERING': 'lightpink'}

dash = pd.read_csv("/home/fabric/work/DASH_BUFFER_LOG.csv")
dash = dash.loc[dash.CurrentPlaybackState.isin(c.keys() )]
states = pd.DataFrame({'startState': dash.CurrentPlaybackState[0:-2].values, 'startTime': dash.EpochTime[0:-2].values,
                        'endState':  dash.CurrentPlaybackState[1:-1].values, 'endTime':   dash.EpochTime[1:-1].values})


for index, s in states.iterrows():
  plt.axvspan(s['startTime'], s['endTime'],  color=c[s['startState']], alpha=1) 

plt.plot(dash[dash.Action!="Writing"].EpochTime, dash[dash.Action!="Writing"].CurrentBufferSize, 'kx:')
plt.title("Buffer(segments)");
plt.xlabel("Time (s)");

## Execute constant bit rate experiment with interruption

For this section, you will need an SSH session on the “router” node and one on the “romeo” node.

In the experiment with constant bit rate, you may not have experienced any rebuffering.

To see how the video client works when there is a temporary interruption in the network, try repeating this experiment, but during the video session, reduce the network data rate to a very low value in middle of the session.

On the “router”, set a constant bit rate of 1000 Kbits/second with

``` bash
bash rate-set.sh 1000Kbit
```

Then, on the client (“romeo”), start the DASH player with the “basic” adaptation policy:

``` bash
python3 ~/AStream/dist/client/dash_client.py -m http://192.168.1.2/media/BigBuckBunny/4sec/BigBuckBunny_4s.mpd -p 'basic' -d
```

Leave this running for a while. Then, on the “router”, reduce the network data rate to 50 Kbits/second:

    bash rate-set.sh 50Kbit

After some time has elapsed, restore the original data rate.

``` bash
bash rate-set.sh 1000Kbit
```

Then, after a little while longer, stop the video client on “romeo” with Ctrl+C.

As before, the logs produced by the client will be located inside a directory named `ASTREAM_LOGS` in your home directory on the “romeo” node. Use

``` bash
ls ~/ASTREAM_LOGS
```

to find these. We will copy the file associated with *this* experiment to `~/ASTREAM_LOGS/DASH_BUFFER_LOG_last.csv` with

``` bash
cp $(ls -t1  ~/ASTREAM_LOGS/DASH_BUFFER_LOG_*  | head -n 1 ) ~/ASTREAM_LOGS/DASH_BUFFER_LOG-last.csv
```

Also reconstruct the video that was delivered to the client. Use

``` bash
suffix=$(ls -lt | grep "TEMP_" | head -n 1 | cut -f2 -d"_")
cd ~/TEMP_$suffix
rm -f ~/BigBuckBunny.mp4 # if it exists
cat BigBuckBunny_4s_init.mp4 $(ls -vx BigBuckBunny_*.m4s) > BigBuckBunny_tmp.mp4
ffmpeg -i  BigBuckBunny_tmp.mp4 -c copy ~/BigBuckBunny.mp4
```

to combine the video segments into a `BigBuckBunny.mp4` file in your home directory.

Then, you can repeat the data analysis steps as before.

## Compare adaptive video policies

For this section, you will need an SSH session on the “router” node and one on the “romeo” node.

As in the previous experiment, in this experiment we will use a constant bit rate, with a brief interruption. However, we will compare the way two video adaptation policies react to this interruption - we’ll compare a *rate based policy* (`basic`) and a *buffer based policy* (`netflix`) under identical settings (similar “high” network rate, “low” data rate, similar duration of the “high” data rate before the interruption, and and similar duration of the “interruption”).

#### Rate-based vs. buffer-based policies

The basic rate adaptation policy (`basic`) chooses a video rate based on the observed download rate. It keeps track of the average download time for recent segments, and calculates a running average.

If the download rate exceeds the current video rate by some threshold, it may increase the video rate. If the download rate is lower than the current video rate, it decreases the video rate to ensure smooth playback.

You can see the source code for the `basic` policy here: [basic_dash2.py](../AStream/dist/client/adaptation/basic_dash2.py)

The buffer-based policy (`netflix`) adapts the video rate based on the current buffer occupancy, rather than the current download rate. When there are many segments already buffered, it can increase the video rate; if the buffer occupancy is low and the client is at risk of rebuffering, it must decrease the video rate.

You can see the source code for the `netflix` policy here: [netflix_dash.py](../AStream/dist/client/adaptation/netflix_dash.py). This policy is based on the paper:

> Te-Yuan Huang, Ramesh Johari, and Nick McKeown. 2013. Downton abbey without the hiccups: buffer-based rate adaptation for HTTP video streaming. In Proceedings of the 2013 ACM SIGCOMM workshop on Future human-centric multimedia networking (FhMN ’13). Association for Computing Machinery, New York, NY, USA, 9–14. https://doi.org/10.1145/2491172.2491179

The policy defines two buffer occupancy thresholds: reservoir (defaults to 10%) and cushion (defaults to 90%). If the buffer occupancy is below the reservoir threshold, it selects the minimum video rate to fill the buffer quickly. If the buffer is within the reservoir and cushion range, it selects a video rate using a rate map function that maps buffer occupancy to video rate according to some increasing function. If the buffer occupancy exceeds the cushion threshold, it selects the maximum video rate.

A brief explanation of the key variables in this policy follows:

-   **Reservoir**: Imagine you have a water tank that supplies water to your house. The “reservoir” in this context is like the water level at the bottom of the tank. It’s the minimum amount of water that needs to be there at all times to ensure a steady and uninterrupted water supply. If the reservoir is too low, you might experience water interruptions. Similarly, in video streaming, the reservoir is the minimum amount of video content that must be stored in the buffer to ensure a smooth playback experience. It’s like having a small reserve of video data to prevent any pauses or disruptions if there are temporary changes in network conditions.
-   **Cushion**: Think of a cushion on a couch. It’s a soft layer that provides comfort and support. The “cushion” in this context is like an extra layer of video content stored in the buffer above the reservoir level. It’s there to give extra protection against sudden changes. Just as a cushion absorbs some impact, this cushion of video content absorbs any fluctuations in network performance. In video streaming, the cushion is an additional amount of video data stored in the buffer beyond the reservoir level. It’s like having a padding of video content that helps maintain a consistent and uninterrupted playback experience, even when there are brief variations in network speed.

Let’s get started!

### Execute the experiment for the rate based policy

On the “router”, set a constant bit rate of 5000 Kbits/second with

``` bash
bash rate-set.sh 5000Kbit
```

Then, on the client (“romeo”), start the DASH player with the “basic” adaptation policy and start the timer along with:

``` bash
python3 ~/AStream/dist/client/dash_client.py -m http://192.168.1.2/media/BigBuckBunny/4sec/BigBuckBunny_4s.mpd -p 'basic' -d
```

Let the DASH player run for 100 seconds. After this duration, on the “router” node, reduce the network data rate to 350 Kbits/second:

``` bash
bash rate-set.sh 350Kbit
```

After an additional 75 seconds (175 seconds total runtime so far), restore the original data rate.

``` bash
bash rate-set.sh 5000Kbit
```

Then, after 300 second, stop the video client on “romeo” with Ctrl+C.

As before, the logs produced by the client will be located inside a directory named `ASTREAM_LOGS` in your home directory on the “romeo” node. Use

``` bash
ls ~/ASTREAM_LOGS
```

to find these. We will copy the file associated with *this* experiment to `~/ASTREAM_LOGS/DASH_BUFFER_LOG_last.csv` with

``` bash
cp $(ls -t1  ~/ASTREAM_LOGS/DASH_BUFFER_LOG_*  | head -n 1 ) ~/ASTREAM_LOGS/DASH_BUFFER_LOG-last.csv
```

Also reconstruct the video that was delivered to the client. Use

``` bash
suffix=$(ls -lt | grep "TEMP_" | head -n 1 | cut -f2 -d"_")
cd ~/TEMP_$suffix
rm -f ~/BigBuckBunny.mp4 # if it exists
cat BigBuckBunny_4s_init.mp4 $(ls -vx BigBuckBunny_*.m4s) > BigBuckBunny_tmp.mp4
ffmpeg -i  BigBuckBunny_tmp.mp4 -c copy ~/BigBuckBunny.mp4
```

to combine the video segments into a `BigBuckBunny.mp4` file in your home directory.

Repeat the data analysis steps as before. Save the figures and the reconstructed video for the rate based policy, so that after completing the experiment in the next section with the buffer based policy, you can compare their behavior.

### Execute the experiment for the buffer based policy

On the “router”, set a constant bit rate of 5000 Kbits/second with

``` bash
bash rate-set.sh 5000Kbit
```

Then, on the client (“romeo”), start the DASH player with the “netflix” adaptation policy and start the timer along with:

``` bash
python3 ~/AStream/dist/client/dash_client.py -m http://192.168.1.2/media/BigBuckBunny/4sec/BigBuckBunny_4s.mpd -p 'netflix' -d
```

Let the DASH player run for 100 seconds. After this duration, on the “router” node, reduce the network data rate to 350 Kbits/second:

``` bash
bash rate-set.sh 350Kbit
```

After an additional 75 seconds (175 seconds total runtime so far), restore the original data rate.

``` bash
bash rate-set.sh 5000Kbit
```

Then, after 300 second, stop the video client on “romeo” with Ctrl+C.

As before, the logs produced by the client will be located inside a directory named `ASTREAM_LOGS` in your home directory on the “romeo” node. Use

``` bash
ls ~/ASTREAM_LOGS
```

to find these. We will copy the file associated with *this* experiment to `~/ASTREAM_LOGS/DASH_BUFFER_LOG_last.csv` with

``` bash
cp $(ls -t1  ~/ASTREAM_LOGS/DASH_BUFFER_LOG_*  | head -n 1 ) ~/ASTREAM_LOGS/DASH_BUFFER_LOG-last.csv
```

Also reconstruct the video that was delivered to the client. Use

``` bash
suffix=$(ls -lt | grep "TEMP_" | head -n 1 | cut -f2 -d"_")
cd ~/TEMP_$suffix
rm -f ~/BigBuckBunny.mp4 # if it exists
cat BigBuckBunny_4s_init.mp4 $(ls -vx BigBuckBunny_*.m4s) > BigBuckBunny_tmp.mp4
ffmpeg -i  BigBuckBunny_tmp.mp4 -c copy ~/BigBuckBunny.mp4
```

to combine the video segments into a `BigBuckBunny.mp4` file in your home directory.

Repeat the data analysis steps as before. Save the figures and the reconstructed video for the buffer based policy, for comparison with the rate based policy in the previous section.

### Discussions

After analyzing the video rate vs. time and buffer vs. time graphs for both the (`basic`) and (`netflix`) adaptation policies, along with the actual video playback, several key observations can be made.

Both policies initially start with increasing video rates as the buffer needs to be filled quickly due to being empty. However, in the rate-based policy, the video rate increases more rapidly compared to the buffer-based policy. In the rate-based policy, once the video rate reaches its maximum value, the video runs smoothly without interruption. On the other hand, in the buffer-based policy, the video rate increases, but small interruptions (not exactly buffering) occur because the video rate has not yet reached its maximum.

During the interruption phase, the differences in behavior between the two policies become more apparent. In the rate-based (`basic`) policy, there is a noticeable decrease in the video rate and buffering events. This is attributed to the policy’s focus on adjusting the video rate based on the observed download rate. As the data rate decreases, the policy responds by reducing the buffer occupancy, resulting in buffering. In contrast, the buffer-based (`netflix`) policy experiences no buffering events during interruptions. This is a direct consequence of the policy’s adaptation strategy, which relies on the current buffer occupancy rather than the download rate. The buffer-based policy ensures sufficient buffer to accommodate interruptions, leading to smooth and uninterrupted playback.

The buffer-based policy (`netflix`) maintains a consistently higher buffer level throughout the experiment, even during interruptions, where it decreases but not significantly. This is evident from the buffer occupancy graph, which generally remains above the buffer threshold level “reservoir”.Conversely, the rate-based policy (`basic`) exhibits fluctuations in buffer occupancy. It begins with a lower buffer level, and during interruptions, the buffer depletes rapidly, resulting in buffering events and drops in video rate.

**Video length**: Both policies were executed for a duration of 300 seconds, yet a notable difference exists in the effective video length delivered to the client. In the buffer-based policy, the delivered video length is closer to the original full video length (approximately 594 seconds), indicating more stable and continuous playback.Conversely, in the rate-based policy, the delivered video length slightly exceeds 300 seconds, highlighting that buffering events and reduced buffer size during interruptions led to incomplete video playback.

## Execute experiment with varying bit rate (mobile user)

Finally, you can try to experience adaptive video as a mobile user!

Repeat the experiment, but instead of setting a constant data rate on the router, you can let it play back a “trace” file with e.g. 

``` bash
bash rate-vary.sh ~/Dataset_1/Dataset/Ferry/Ferry5.csv 0.1
```

where the first argument is the path to a trace file, and the second argument is a scaling factor greater than 0 but less than 1. (The smaller the scaling factor, the lower the network quality while still preserving the trace dynamics.)

The following figure shows the “dynamics” (throughput in Mbps against time) for each of the traces:

![](https://witestlab.poly.edu/blog/content/images/2022/04/nyc-traces.png)

For some traces, the throughput is always more than enough to steam the video at the highest quality level. For the traces where the throughput is *not* sufficient to stream continuously at the highest quality level, a good decision policy should still be able to smooth over the variation in network quality and deliver high quality video without rebuffering.

While playing back a trace on the “router”, on the client (“romeo”), start the DASH player with the “basic” adaptation policy:

``` bash
python3 ~/AStream/dist/client/dash_client.py -m http://192.168.1.2/media/BigBuckBunny/4sec/BigBuckBunny_4s.mpd -p 'basic' -d
```

Leave this running for a while. Then, stop the video client on “romeo” with Ctrl+C.

As before, the logs produced by the client will be located inside a directory named `ASTREAM_LOGS` in your home directory on the “romeo” node. Use

``` bash
ls ~/ASTREAM_LOGS
```

to find these. We will copy the file associated with *this* experiment to `~/ASTREAM_LOGS/DASH_BUFFER_LOG_last.csv` with

``` bash
cp $(ls -t1  ~/ASTREAM_LOGS/DASH_BUFFER_LOG_*  | head -n 1 ) ~/ASTREAM_LOGS/DASH_BUFFER_LOG-last.csv
```

Also reconstruct the video that was delivered to the client. Use

``` bash
suffix=$(ls -lt | grep "TEMP_" | head -n 1 | cut -f2 -d"_")
cd ~/TEMP_$suffix
rm -f ~/BigBuckBunny.mp4 # if it exists
cat BigBuckBunny_4s_init.mp4 $(ls -vx BigBuckBunny_*.m4s) > BigBuckBunny_tmp.mp4
ffmpeg -i  BigBuckBunny_tmp.mp4 -c copy ~/BigBuckBunny.mp4
```

to combine the video segments into a `BigBuckBunny.mp4` file in your home directory.

Then, you can repeat the data analysis steps as before.

## Exercises

After you have run the experiment, answer the following questions:

-   In constant bit rate experiment, did you observe any buffering in any of the DASH policies? Explain.
-   In constant bit rate with interruptions experiment, did you see any buffering in basic, netflix and sara policies? Explain with the screenshots from your analysis. Also comment on how well each policy recovers from the buffering and which policy is most resilient to interruptions.
-   In varying bit rate experiment, which of the DASH policy has better average video rate? Justify using the screenshots.

The following questions requires you to modify certain parameters in each of the experiment:

-   What happens when you increase the network capacity from 1Mbps to 4Mbps in constant bit rate experiment?

Go to the client node rome and change the current directory to edit the config_dash.py file by running the following commands:

``` bash
cd ~/AStream/dist/client
nano config_dash.py
```

Under the SARA section, change ALPHA_BUFFER_COUNT to 10 and BETA_BUFFER_COUNT to 15 and Ctrl+X, 'Y' for Yes and 'Enter' to save the changes and exit the file. Now perform the constant bit rate using SARA policy.

Note: Revert the configuration changes back to initial settings(ALPHA_BUFFER_COUNT = 5 and BETA_BUFFER_COUNT = 10) by editing config_dash.py file after analysing the logs.

-   Observe and explain how changing the buffer thresholds effects different metrics such as average video rate and number of bitrate switching events?

Perform varying bit rate experiment with scaling factor of 0.25 for each DASH policy. On router node, run:

``` bash
bash rate-vary.sh ~/Dataset_1/Dataset/Ferry/Ferry5.csv 0.25
```

-   What differences do you see in terms of different metrics by changing the scaling factor from 0.1 to 0.25 for each DASH policy?

## Write your own adaptive video policy

Do you think you can improve on these adaptive video policies?

To write you own adaptive video policy, open basic_dash2.py file that was cloned to your jupyter environment by clicking this link:

[AStream/dist/client/adaptation/basic_dash2.py](AStream/dist/client/adaptation/basic_dash2.py)

Based on your observations from before, make your own changes to this policy and save it.

SCP locally modified policy file to your client romeo such that it replaces previous basic_dash2.py file inside adaptive-video/AStream/dist/client/adaptation/ location.

Re-run constant bit rate, constant bit rate with interruption and mobile user experiments to analyze the performance of your policy.